# Cleanup

This Jupyter notebook is the data cleaning pipeline to prepare the raw data from the Mongo Database to be used for the data analysis and the data visualisation on the dashboard. 

This script requires the following packages:

In [3]:
import sys
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib
from matplotlib import pyplot as plt
import config

## Fetching the Data

In [6]:
client = MongoClient(config.mongo_uri)
db = client.smellStation
raw_data = db.sensorData

In [7]:
df = pd.DataFrame(list(raw_data.find()))


In [8]:
df

,_id,occupancy,bin1,bin2,H2S,temperature,humidity,timestamp
0,61ae4fc31a0b71deea1fb5db,30,29.51,0.00,866,26.3,60.0,1638813600
1,61ae521c1a0b71deea1fb5dc,27,28.97,0.00,902,28.1,35.0,1638814200
2,61ae54741a0b71deea1fb5dd,26,31.55,37.78,929,29.2,31.0,1638814800
3,61ae589e1a0b71deea1fb5de,25,-1.00,44.11,-1,-1.0,-1.0,1638815402
4,61af24849ace796ec57705b8,-1,27.81,0.00,915,26.7,40.0,1638867600
...,...,...,...,...,...,...,...,...
956,61b7e3e7462ce0cb986aebf6,-1,29.49,13.19,-1,-1.0,-1.0,1639441202
957,61b7e63f462ce0cb986aebf7,-1,31.29,25.85,-1,-1.0,-1.0,1639441802
958,61b7e897462ce0cb986aebf8,-1,32.22,13.95,-1,-1.0,-1.0,1639442402
959,61b7eaef462ce0cb986aebf9,-1,35.18,29.37,-1,-1.0,-1.0,1639443002
